In [1]:
%pip install -e 'src/hear21passt'
%pip install pydub

Obtaining file:///root/data/exploratory_notebooks/da/src/hear21passt
  Preparing metadata (setup.py) ... done
  Attempting uninstall: hear21passt
    Found existing installation: hear21passt 0.0.23
    Uninstalling hear21passt-0.0.23:
      Successfully uninstalled hear21passt-0.0.23
  Running setup.py develop for hear21passt

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorboard


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [1]:
import sys
import os
import datetime
import re

import numpy as np
import pandas as pd
import torch
import torchaudio
import tqdm

from operator import itemgetter
from hashlib import sha1

from torch.optim import AdamW, lr_scheduler as LR
from torch.utils.data import default_collate
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ConcatDataset, WeightedRandomSampler

from sklearn.metrics import f1_score

from IPython.display import Audio

from optim.lr_scheduler import get_scheduler_lambda, ReduceLROnPlateauHumanized
from utils.model import get_pretrained_passt_model
from utils.gdsc22_dataset_legacy import BuzzMapDataset, BuzzDatasetWeighter, BuzzMapTestDataset
from utils.dataset_augmentations import (
    BuzzMapTransformedDataset, 
    BuzzMapTransformedTestDataset,
    BuzzTorchDataLoader,
    MixUp1Level,
    MixUp2Level, 
    RandomGain,
    Rolling,
    ColoredNoise,
    NormalizeAmplitudes
)

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 1. Initial configuration

In [2]:
torch.manual_seed(42)
np.random.seed(42)

In [3]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_processed_dir = os.path.join(project_dir, 'gdsc_data/data_processed/')
data_source_dir = os.path.join(project_dir, 'gdsc_data/data_source/')

In [4]:
ts = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

MODEL_SAMPLING_RATE = 32000

MODEL_NAME = f"gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_long_short_samples_resampled_epochs10_transformation=[normed]@{ts}.pt"
TORCH_SAVE_PATH = os.path.join(project_dir, f'checkpoints/{MODEL_NAME}')

TENSORBOARD_RUNS_PATH = os.path.join(project_dir, "torch_runlogs/")
TENSORBOARD_EXP_FOLDER = f"gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_long_short_samples_resampled_epochs10_transformation=[normed]@{ts}"
TENSORBOARD_EXP_COMMENT = "PaSST_s_kd_p16_128_ap486_NOEXP_lre_mean_loss_train+val_stratified_long_short_samples_resampled_epochs10_transformation=[normed]"

TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32

# fine tuning and ohe of real labels
CLASSIFIER_N_CLASSES = 66

DATA_TAG = "data_8c86715"

DEBUG_TRANSFORMATIONS = False

In [5]:
#final_dataset_path = find_final_dataset_path(data_processed_dir, DATA_TAG)
final_dataset_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '01_apply_ir_function')
final_dataset_val_as_train_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '03_apply_ir_function_on_validation')

final_dataset_path_short = os.path.join(data_processed_dir, DATA_TAG, '07_split_original_into_windows_with_short_samples/')
final_dataset_path_long = os.path.join(data_processed_dir, DATA_TAG, '07_split_original_into_windows_with_short_samples/')

final_dataset_val_as_train_path_short = os.path.join(data_processed_dir, DATA_TAG, '08_spit_resampled_validation_into_windows_with_short_samples/')
final_dataset_val_as_train_path_long = os.path.join(data_processed_dir, DATA_TAG, '08_spit_resampled_validation_into_windows_with_short_samples')

final_dataset_val_as_train_path = os.path.join(data_processed_dir, DATA_TAG, '05_split_resampled_validation_into_windows/')

In [6]:
train_dir_augmented = os.path.join(final_dataset_path_augmented, 'train/')
val_as_train_dir_augmented = os.path.join(final_dataset_val_as_train_path_augmented, 'train/')

train_dir_short = os.path.join(final_dataset_path_short, 'train/short_only')
train_dir_long = os.path.join(final_dataset_path_short, 'train/long_only')
val_as_train_dir_short = os.path.join(final_dataset_val_as_train_path_short, 'train/short_only')
val_as_train_dir_long = os.path.join(final_dataset_val_as_train_path_long, 'train/long_only')
new_sampled_val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')

val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')
#test_dir = os.path.join(final_dataset_test_path, 'test/')

## 2. Dataloaders and batching

### 2.1 Debugging collate_fn

Used to check the random seed and batchability of the individual points 0:)

In [7]:
def debug_collate(batch):
    get_1st = itemgetter(0)
    files = ", ".join(map(get_1st, batch)).encode('utf-8')
    worker_info = torch.utils.data.get_worker_info()   
    print(f"Worker_id '{worker_info.id}' batch hash is '{sha1(files).hexdigest()}'")
    
    batch = default_collate(batch)
    return batch

In [8]:
train_ds_short = BuzzMapDataset(
    train_dir_short,
    metadata_csv=os.path.join(train_dir_short, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES,
)

train_ds_long = BuzzMapDataset(
    train_dir_long,
    metadata_csv=os.path.join(train_dir_long, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES,
)

val_ds_short = BuzzMapDataset(
    val_as_train_dir_short,
    metadata_csv=os.path.join(val_as_train_dir_short, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES
)

val_ds_long = BuzzMapDataset(
    val_as_train_dir_long,
    metadata_csv=os.path.join(val_as_train_dir_long, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES
)

new_val_dataset = BuzzMapDataset(
    new_sampled_val_dir,
    metadata_csv=os.path.join(new_sampled_val_dir, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES    
)

# test_dataset = BuzzMapTestDataset(
#     test_dir,
#     metadata_csv=os.path.join(val_as_train_dir, 'metadata.csv'),
#     num_classes=CLASSIFIER_N_CLASSES
# )

train_dataset_short = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS), 
        #RandomGain(debug=DEBUG_TRANSFORMATIONS), 
        #Rolling(shift_sec=0.5, SR=MODEL_SAMPLING_RATE, debug=DEBUG_TRANSFORMATIONS), 
        #MixUp1Level(debug=DEBUG_TRANSFORMATIONS), 
        #ColoredNoise(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=train_ds_short,
    #debug=True
)

train_dataset_long = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=train_ds_long,
    #debug=True
)

val_dataset_short = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=val_ds_short,
    #debug=True
)

val_dataset_long = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS), 
        #RandomGain(debug=DEBUG_TRANSFORMATIONS), 
        #Rolling(shift_sec=0.5, SR=MODEL_SAMPLING_RATE, debug=DEBUG_TRANSFORMATIONS), 
        #MixUp1Level(debug=DEBUG_TRANSFORMATIONS), 
        #ColoredNoise(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=val_ds_long,
    #debug=True
)

new_val_dataset_normed = BuzzMapTransformedDataset(
    transformers=[
        NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
    ],
    buzz_mappable=new_val_dataset,
    #debug=True
)

# test_dataset_normed = BuzzMapTransformedTestDataset(
#     transformers=[
#         NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)
#     ],
#     buzz_mappable=test_dataset,
#     #debug=True    
# )

# I had too much hope on the IR augmented dataset and took a wrong path...
# train_dataset_augmented = BuzzAugmentedIterableDataset(
#     train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )
# val_as_train_dataset_augmented = BuzzAugmentedIterableDataset(
#     val_as_train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_val_as_train_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )

dataset_weights_train_val, num_samples = BuzzDatasetWeighter()(train_ds_short, train_ds_long, val_ds_short, val_ds_long)

# train_dataloader_normed = BuzzTorchDataLoader(
#     train_dataset_normed, 
#     batch_size=TRAIN_BATCH_SIZE, 
#     pin_memory=True,
#     shuffle=True,
#     num_workers=2,
#     prefetch_factor=2,
#     worker_init_fn=train_dataset_normed._init_multiprocess_worker,    
# )
    
train_dataloader = BuzzTorchDataLoader(
    ConcatDataset([train_ds_short, train_ds_long, val_ds_short, val_ds_long]),
    batch_size=TRAIN_BATCH_SIZE, 
    pin_memory=True,
    sampler=WeightedRandomSampler(dataset_weights_train_val, num_samples),
    num_workers=3,
    prefetch_factor=2,
    worker_init_fn=new_val_dataset_normed._init_multiprocess_worker,
    #collate_fn=debug_collate
)

# val_dataloader = BuzzTorchDataLoader(
#     val_dataset_normed, 
#     batch_size=VAL_BATCH_SIZE, 
#     pin_memory=True,
#     shuffle=True,
#     num_workers=2,
#     prefetch_factor=2,
#     worker_init_fn=train_dataset_normed._init_multiprocess_worker,        
# )

new_val_dataloader = BuzzTorchDataLoader(
    new_val_dataset_normed, 
    batch_size=VAL_BATCH_SIZE, 
    pin_memory=True,
    shuffle=True,
    num_workers=2,
    prefetch_factor=2,
    worker_init_fn=new_val_dataset_normed._init_multiprocess_worker,            
)

## 3. Init pytorch objects

In [9]:
writer = SummaryWriter(os.path.join(TENSORBOARD_RUNS_PATH, TENSORBOARD_EXP_FOLDER), comment=TENSORBOARD_EXP_COMMENT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
%%sh
nvidia-smi

Sun Jul 16 19:50:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
# Default stride for t and f is 10
# NOTE: FMAX is used in frequency augmentation. Every sample has a random subset of fmin and fmax applied to it...
model = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=40, s_patchout_f=4)

#model = get_pretrained_passt_model(arch='passt_s_swa_p16_128_ap476', mode="all", n_classes=66, s_patchout_t=10, s_patchout_f=3)
model.to(device)



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

PaSSTGDSCIface(
  (mel): AugmentMelSTFT(
    winsize=800, hopsize=320
    (freqm): FrequencyMasking()
    (timem): TimeMasking()
  )
  (net): PaSST(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=307

In [ ]:
adamw = AdamW(model.net.parameters(), lr=0.00001, weight_decay=0.0001)
#lr_scheduler = torch.optim.lr_scheduler.LambdaLR(adamw, get_scheduler_lambda(warm_up_len=0))
lr_scheduler = LR.StepLR(adamw, step_size=10, gamma=0.5)
scaler = torch.cuda.amp.GradScaler()

TORCH_SAVE_FILE, EXT = os.path.splitext(TORCH_SAVE_PATH)

train_loop_dataloader = train_dataloader
val_loop_dataloader = new_val_dataloader

epoch_begin = 0
epoch_count = 20

print("=" * 80)
print(f"STEP1: {epoch_count}epochs using {train_loop_dataloader}\r\n")
print(f"Val using {val_loop_dataloader}\r\n")
print(f"Initial Learning rate:{lr_scheduler.get_last_lr()}\r\n")
print("=" * 80)


LOSS_REDUCTION_METHOD = 'mean'


for epoch in range(epoch_begin, epoch_count):
    y_tr_pred_f1 = []
    y_tr_true_f1 = []
    train_loss = []

    model.train()
    for _, _, X_batch, y in (progress := tqdm.tqdm(train_loop_dataloader)):
        adamw.zero_grad()        
        
        X_batch = X_batch.to(device)
        y = y.to(device)
        X_batch_spectr = model.mel(X_batch).unsqueeze(1)
        
        with  torch.cuda.amp.autocast(): 
            y_hat, emb = model.net(X_batch_spectr)     
            
            loss = F.cross_entropy(y_hat, y, reduction="none")
            loss_reduce_method = getattr(loss, LOSS_REDUCTION_METHOD)
            loss = loss_reduce_method()            
            
            train_loss.append(loss.detach().cpu())
            progress.set_description(f"TrainL {float(loss):.2f}")
            
        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()
        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(adamw)
        # Updates the scale for next iteration.
        scaler.update()
        
        prob = F.softmax(y_hat.detach(), dim=1)
        y_pred = torch.argmax(prob.detach(), dim=1)
        y_tr_pred_f1.append(y_pred.cpu())
        y_tr_true_f1.append(y.cpu().argmax(dim=1))

    f_score_train = f1_score(torch.cat(y_tr_true_f1).numpy(), torch.cat(y_tr_pred_f1).numpy(), average='macro')
    total_train_loss = torch.stack(train_loss, dim=0)
    total_train_reduction_method = getattr(total_train_loss, LOSS_REDUCTION_METHOD)
    total_train_loss = total_train_reduction_method()
    print(f"=" * 80)
    print("\r\n")
    print(f"Train Loss(epoch={epoch}): {total_train_loss}")
    print(f"F1 score(train) {f_score_train}")
    print("\r\n")
    print(f"=" * 80)
    print("\r\n")

    val_loss = []
    y_val_pred_f1 = []
    y_val_true_f1 = []
    model.eval()
    
    with torch.no_grad():
        for _, _, _, X_batch, y_val in (progress := tqdm.tqdm(val_loop_dataloader)):
            X_batch = X_batch.to(device)
            y_val = y_val.to(device)
            X_batch_spectr = model.mel(X_batch).unsqueeze(1)

            with torch.cuda.amp.autocast():
                y_hat, emb = model.net(X_batch_spectr)
            
                _loss = F.cross_entropy(y_hat, y_val, reduction="none")
                _loss_reduce_method = getattr(_loss, LOSS_REDUCTION_METHOD)
                _loss = _loss_reduce_method()
                val_loss.append(_loss.detach().cpu())
            
            progress.set_description(f"ValL {float(_loss):.2f}")            

            prob = F.softmax(y_hat.detach(), dim=1)
            y_pred = torch.argmax(prob.detach(), dim=1)

            y_val_pred_f1.append(y_pred.cpu())
            y_val_true_f1.append(y_val.cpu().argmax(dim=1))
            
    f1_score_val = f1_score(torch.cat(y_val_true_f1).numpy(), torch.cat(y_val_pred_f1).numpy(), average='macro')
    total_val_loss = torch.stack(val_loss, dim=0)
    total_val_loss_reduction_method = getattr(total_val_loss, LOSS_REDUCTION_METHOD)
    total_val_loss = total_val_loss_reduction_method()
    print(f"=" * 80)
    print("\r\n")
    print(f"Val Loss: {total_val_loss}")
    print(f"F1 score(val) {f1_score_val}")
    print("\r\n")    
    print(f"=" * 80)
    print("\r\n")
          
    writer.add_scalar('Loss/train', total_train_loss, epoch)
    writer.add_scalar('Loss/val', total_val_loss, epoch)
    writer.add_scalar('F1/train', f_score_train, epoch)
    writer.add_scalar('F1/val', f1_score_val, epoch)
    writer.add_scalar('LR', lr_scheduler.get_last_lr()[0], epoch)
    
    lr_scheduler.step()        

    print(f"New lr: {lr_scheduler.get_last_lr()[0]}")
       
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': adamw.state_dict(),
            'lr_scheduler_state_dict': lr_scheduler.state_dict(),
            'grad_scaler_state_dict': scaler.state_dict(),
        }, f"{TORCH_SAVE_FILE}_epoch={epoch}_epochs=[{epoch_begin}, {epoch_begin+epoch_count}]{EXT}")
    
    
# print(f"STEP2: 20epochs on scaled + 'transformed' data")    
# for epoch in range(20, 40):
#     y_tr_pred_f1 = []
#     y_tr_true_f1 = []
#     train_loss = []

#     model.train()
#     for _, _, _, X_batch, y in (progress := tqdm.tqdm(train_dataloader_transformed)):
#         adamw.zero_grad()        
        
#         X_batch = X_batch.to(device)
#         y = y.to(device)
#         X_batch_spectr = model.mel(X_batch).unsqueeze(1)
        
#         with  torch.cuda.amp.autocast(): 
#             y_hat, emb = model.net(X_batch_spectr)     
            
#             loss = F.cross_entropy(y_hat, y, reduction="none")
#             loss = loss.mean()
            
#             train_loss.append(loss.detach().cpu())
#             progress.set_description(f"TrainL {float(loss):.2f}")
            
#         # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
#         # Backward passes under autocast are not recommended.
#         # Backward ops run in the same dtype autocast chose for corresponding forward ops.
#         scaler.scale(loss).backward()
#         # scaler.step() first unscales the gradients of the optimizer's assigned params.
#         # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
#         # otherwise, optimizer.step() is skipped.
#         scaler.step(adamw)
#         # Updates the scale for next iteration.
#         scaler.update()
        
#         prob = F.softmax(y_hat.detach(), dim=1)
#         y_pred = torch.argmax(prob.detach(), dim=1)
#         y_tr_pred_f1.append(y_pred.cpu())
#         y_tr_true_f1.append(y.cpu().argmax(dim=1))

#     f_score_train = f1_score(torch.cat(y_tr_true_f1).numpy(), torch.cat(y_tr_pred_f1).numpy(), average='macro')
#     total_train_loss = torch.stack(train_loss, dim=0).mean()
#     print(f"=" * 80)
#     print("\r\n")
#     print(f"Train Loss(epoch={epoch}): {total_train_loss}")
#     print(f"F1 score(train) {f_score_train}")
#     print("\r\n")
#     print(f"=" * 80)
#     print("\r\n")

#     val_loss = []
#     y_val_pred_f1 = []
#     y_val_true_f1 = []
#     model.eval()
    
#     with torch.no_grad():
#         for _, _, _, X_batch, y_val in (progress := tqdm.tqdm(val_dataloader_normed)):
#             X_batch = X_batch.to(device)
#             y_val = y_val.to(device)
#             X_batch_spectr = model.mel(X_batch).unsqueeze(1)

#             with torch.cuda.amp.autocast():
#                 y_hat, emb = model.net(X_batch_spectr)
            
#                 _loss = F.cross_entropy(y_hat, y_val, reduction="none")
#                 _loss = _loss.mean()
#                 val_loss.append(_loss.detach().cpu())
            
#             progress.set_description(f"ValL {float(_loss):.2f}")            

#             prob = F.softmax(y_hat.detach(), dim=1)
#             y_pred = torch.argmax(prob.detach(), dim=1)

#             y_val_pred_f1.append(y_pred.cpu())
#             y_val_true_f1.append(y_val.cpu().argmax(dim=1))
            
#     f1_score_val = f1_score(torch.cat(y_val_true_f1).numpy(), torch.cat(y_val_pred_f1).numpy(), average='macro')
#     total_val_loss = torch.stack(val_loss, dim=0).mean()
#     print(f"=" * 80)
#     print("\r\n")
#     print(f"Val Loss: {total_val_loss}")
#     print(f"F1 score(val) {f1_score_val}")
#     print("\r\n")    
#     print(f"=" * 80)
#     print("\r\n")
       
#     writer.add_scalar('Loss/train', total_train_loss, epoch)
#     writer.add_scalar('Loss/val', total_val_loss, epoch)
#     writer.add_scalar('F1/train', f_score_train, epoch)
#     writer.add_scalar('F1/val', f1_score_val, epoch)
#     writer.add_scalar('LR', lr_scheduler.get_last_lr()[0], epoch)

#     lr_scheduler.step()
#     print(f"New lr: {lr_scheduler.get_last_lr()[0]}")
       
#     torch.save(
#         {
#             'epoch': epoch,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': adamw.state_dict(),
#             'grad_scaler_state_dict': scaler.state_dict(),
#         }, f"{TORCH_SAVE_FILE}_2nd20epochs(transformed_dataset){EXT}")   

STEP1: 20epochs using <torch.utils.data.dataset.ConcatDataset object at 0x7f61b67f55a0>

Val using <class 'utils.dataset_augmentations.BuzzMapTransformedDataset'>: Transformations=(<utils.dataset_augmentations.NormalizeAmplitudes object at 0x7f61b67f7f70>); Mapper=(<class 'utils.gdsc22_dataset_legacy.BuzzMapDataset'>: reading data from /root/data/gdsc_data/data_processed/data_8c86715/05_split_resampled_validation_into_windows/val/metadata.csv. Dataset stats: shape=(358, 3))

Initial Learning rate:[1e-05]



  0%|          | 0/717 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([32, 1, 128, 1000])
self.norm(x) torch.Size([32, 768, 12, 99])
 patch_embed :  torch.Size([32, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X Before time Patchout of 40  torch.Size([32, 768, 12, 99])
X after time Patchout torch.Size([32, 768, 12, 59])
X Before Freq Patchout of 4  torch.Size([32, 768, 12, 59])
 
 X after freq Patchout:  torch.Size([32, 768, 8, 59])
X flattened torch.Size([32, 472, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([32, 1, 768])
 self.dist_token.shape torch.Size([32, 1, 768])
 final sequence x torch.Size([32, 474, 768])
 after 12 atten blocks x torch.Size([32, 474, 768])
forward_features torch.Size([32, 768])
head torch.Size([32, 66])


TrainL 1.96:  84%|████████▎ | 600/717 [09:29<01:51,  1.05it/s]

In [17]:
!ls -ahl /root/data/checkpoints/

total 82G
drwxr-xr-x  2 root root     22K Jul 16 18:20  .
drwx------ 21 root nogroup 6.0K Jul 16 13:52  ..
-rw-r--r--  1 root root    978M Jul 16 16:54 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T16:44:23_epoch=0_epochs=[0, 10].pt'
-rw-r--r--  1 root root    978M Jul 16 17:03 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T16:44:23_epoch=1_epochs=[0, 10].pt'
-rw-r--r--  1 root root    978M Jul 16 17:13 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T16:44:23_epoch=2_epochs=[0, 10].pt'
-rw-r--r--  1 root root    978M Jul 16 17:22 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T16:44:23_epoch=3_epochs=[0, 10].pt'
-rw-r--r--  1 root root    978M Jul 16 17:32 'gdsc22_passt_kd_NOEXP_lr_mismatch_s

## Finetune model trained on first 40 epochs

In [11]:
TORCH_LOAD_FILE = os.path.join(project_dir, 'checkpoints', 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_fixed_shuffle_mean_loss_epochs40_transformation=[normed]@2023-07-15T12:24:34_epochs=[0, 40].pt')

model = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=40, s_patchout_f=4)
state = torch.load(TORCH_LOAD_FILE, map_location=torch.device('cpu'))
print(model.load_state_dict(state['model_state_dict']))

model.to(device)



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

PaSSTGDSCIface(
  (mel): AugmentMelSTFT(
    winsize=800, hopsize=320
    (freqm): FrequencyMasking()
    (timem): TimeMasking()
  )
  (net): PaSST(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=307

In [12]:
%%sh
nvidia-smi

Sat Jul 15 19:08:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   50C    P0    29W /  70W |    924MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
adamw = AdamW(model.net.parameters(), lr=0.00001, weight_decay=0.0001)
adamw.load_state_dict(state['optimizer_state_dict'])
scaler = torch.cuda.amp.GradScaler()
scaler.load_state_dict(state['grad_scaler_state_dict'])

# kept default for the model @2023-07-15T16:45:42
lr_scheduler = ReduceLROnPlateauHumanized(adamw)
lr_scheduler.load_state_dict(state['lr_scheduler_state_dict'])

ts2 = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
TORCH_SAVE_FILE, EXT = os.path.splitext(TORCH_LOAD_FILE)


train_loop_dataloader = val_dataloader_normed
val_loop_dataloader = new_val_dataloader_normed

epoch_begin = state['epoch'] + 1
epoch_count = 20 
epoch_end = epoch_begin + epoch_count


print("=" * 80)
print(f"STEP2: Additional {epoch_count}epochs ({epoch_begin}-{epoch_end}) using {train_loop_dataloader}\r\n")
print(f"Val using {val_loop_dataloader}\r\n")
print(f"Initial Learning rate:{lr_scheduler.get_last_lr()}\r\n")
print("=" * 80)


LOSS_REDUCTION_METHOD = 'mean'

def loss_reduce_fn(loss, method):
    _reduce_fn = getattr(loss, method)
    return _reduce_fn()


for epoch in range(epoch_begin, epoch_begin + epoch_count):
    y_tr_pred_f1 = []
    y_tr_true_f1 = []
    train_loss = []

    model.train()
    for _, _, _, X_batch, y in (progress := tqdm.tqdm(train_loop_dataloader)):
        adamw.zero_grad()        
        
        X_batch = X_batch.to(device)
        y = y.to(device)
        X_batch_spectr = model.mel(X_batch).unsqueeze(1)
        
        with  torch.cuda.amp.autocast(): 
            y_hat, emb = model.net(X_batch_spectr)     
            
            loss = F.cross_entropy(y_hat, y, reduction="none")
            loss_reduce_method = getattr(loss, LOSS_REDUCTION_METHOD)
            loss = loss_reduce_method()
            
            train_loss.append(loss.detach().cpu())
            progress.set_description(f"TrainL {float(loss):.2f}")
            
        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()
        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(adamw)
        # Updates the scale for next iteration.
        scaler.update()
        
        prob = F.softmax(y_hat.detach(), dim=1)
        y_pred = torch.argmax(prob.detach(), dim=1)
        y_tr_pred_f1.append(y_pred.cpu())
        y_tr_true_f1.append(y.cpu().argmax(dim=1))

    f_score_train = f1_score(torch.cat(y_tr_true_f1).numpy(), torch.cat(y_tr_pred_f1).numpy(), average='macro')
    total_train_loss = torch.stack(train_loss, dim=0)
    total_train_reduction_method = getattr(total_train_loss, LOSS_REDUCTION_METHOD)
    total_train_loss = total_train_reduction_method()
    print(f"=" * 80)
    print("\r\n")
    print(f"Train Loss(epoch={epoch}): {total_train_loss}")
    print(f"F1 score(train) {f_score_train}")
    print("\r\n")
    print(f"=" * 80)
    print("\r\n")

    val_loss = []
    y_val_pred_f1 = []
    y_val_true_f1 = []
    model.eval()
    
    with torch.no_grad():
        for _, _, _, X_batch, y_val in (progress := tqdm.tqdm(val_loop_dataloader)):
            X_batch = X_batch.to(device)
            y_val = y_val.to(device)
            X_batch_spectr = model.mel(X_batch).unsqueeze(1)

            with torch.cuda.amp.autocast():
                y_hat, emb = model.net(X_batch_spectr)
            
                _loss = F.cross_entropy(y_hat, y_val, reduction="none")
                _loss_reduce_method = getattr(_loss, LOSS_REDUCTION_METHOD)
                _loss = _loss_reduce_method()
                val_loss.append(_loss.detach().cpu())
            
            progress.set_description(f"ValL {float(_loss):.2f}")            

            prob = F.softmax(y_hat.detach(), dim=1)
            y_pred = torch.argmax(prob.detach(), dim=1)

            y_val_pred_f1.append(y_pred.cpu())
            y_val_true_f1.append(y_val.cpu().argmax(dim=1))
            
    f1_score_val = f1_score(torch.cat(y_val_true_f1).numpy(), torch.cat(y_val_pred_f1).numpy(), average='macro')
    total_val_loss = torch.stack(val_loss, dim=0)
    total_val_loss_reduction_method = getattr(total_val_loss, LOSS_REDUCTION_METHOD)
    total_val_loss = total_val_loss_reduction_method()
    print(f"=" * 80)
    print("\r\n")
    print(f"Val Loss: {total_val_loss}")
    print(f"F1 score(val) {f1_score_val}")
    print("\r\n")    
    print(f"=" * 80)
    print("\r\n")
       
    writer.add_scalar('Loss/train', total_train_loss, epoch)
    writer.add_scalar('Loss/val', total_val_loss, epoch)
    writer.add_scalar('F1/train', f_score_train, epoch)
    writer.add_scalar('F1/val', f1_score_val, epoch)
    writer.add_scalar('LR', lr_scheduler.get_last_lr(), epoch)

    lr_scheduler.step(f1_score_val)
    print(f"New lr: {lr_scheduler.get_last_lr()}")
       
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': adamw.state_dict(),
            'lr_scheduler_state_dict': lr_scheduler.state_dict(),
            'grad_scaler_state_dict': scaler.state_dict(),
        }, f"{TORCH_SAVE_FILE}_added_training_on_val_loss_sum_epochs=[{epoch_begin}, {epoch_begin+epoch_count}]@{ts2}{EXT}")

STEP2: Additional 20epochs (40-60) using <class 'utils.dataset_augmentations.BuzzMapTransformedDataset'>: Transformations=(<utils.dataset_augmentations.NormalizeAmplitudes object at 0x7ff3ef56acb0>); Mapper=(<class 'utils.gdsc22_dataset_legacy.BuzzMapDataset'>: reading data from /root/data/gdsc_data/data_processed/data_8c86715/05_split_resampled_validation_into_windows/train/metadata.csv. Dataset stats: shape=(2160, 3))

Val using <class 'utils.dataset_augmentations.BuzzMapTransformedDataset'>: Transformations=(<utils.dataset_augmentations.NormalizeAmplitudes object at 0x7ff3ef56ac80>); Mapper=(<class 'utils.gdsc22_dataset_legacy.BuzzMapDataset'>: reading data from /root/data/gdsc_data/data_processed/data_8c86715/05_split_resampled_validation_into_windows/val/metadata.csv. Dataset stats: shape=(358, 3))

Initial Learning rate:2.5e-06



  0%|          | 0/68 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([32, 1, 128, 1000])
self.norm(x) torch.Size([32, 768, 12, 99])
 patch_embed :  torch.Size([32, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X Before time Patchout of 40  torch.Size([32, 768, 12, 99])
X after time Patchout torch.Size([32, 768, 12, 59])
X Before Freq Patchout of 4  torch.Size([32, 768, 12, 59])
 
 X after freq Patchout:  torch.Size([32, 768, 8, 59])
X flattened torch.Size([32, 472, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([32, 1, 768])
 self.dist_token.shape torch.Size([32, 1, 768])
 final sequence x torch.Size([32, 474, 768])
 after 12 atten blocks x torch.Size([32, 474, 768])
forward_features torch.Size([32, 768])
head torch.Size([32, 66])


TrainL 8.61: 100%|██████████| 68/68 [01:01<00:00,  1.11it/s] 




Train Loss(epoch=40): 1023.9627685546875
F1 score(train) 0.7795568786631606






...



Val Loss: 78.5841064453125
F1 score(val) 0.8669523016741669




New lr: 6.25e-07


  0%|          | 0/68 [00:00<?, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
TrainL 2.53: 100%|██████████| 68/68 [00:58<00:00,  1.17it/s] 




Train Loss(epoch=59): 388.72918701171875
F1 score(train) 0.9072773314257395






  0%|          | 0/12 [00:00<?, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
ValL 0.74: 100%|██████████| 12/12 [00:13<00:00,  1.16s/it]




Val Loss: 77.09075927734375
F1 score(val) 0.8669523016741669




New lr: 6.25e-07


In [13]:
%%sh
nvidia-smi

Sat Jul 15 06:16:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   34C    P0    26W /  70W |  14936MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
model_eval = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=0, s_patchout_f=0)
state = torch.load(TORCH_SAVE_PATH)
model_eval.load_state_dict(state['model_state_dict'])



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

FileNotFoundError: [Errno 2] No such file or directory: '/root/data/checkpoints/gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_fixed_shuffle_epochs20_transformation=[normed]_epochs20_transformations=[normed; gain; roll; mixup; color_noise]@2023-07-15T06:07:22.pt'

In [7]:
model_eval = model_eval.cuda()

In [8]:
    val_loss = 0.0
    ypred_f1 = []
    ytrue_f1 = []
    model_eval.eval()
    
    with torch.no_grad():
        for batch, y_val in tqdm.tqdm(val_dataloader):
            batch = batch.cuda()
            y_val = y_val.cuda()

            y_hat, emb = model_eval(batch)
            _loss = F.cross_entropy(y_hat, y_val, reduction="none")

            prob = F.softmax(y_hat, dim=1)
            y_pred = torch.argmax(prob, dim=1)

            ypred_f1.append(y_pred.cpu())
            ytrue_f1.append(y_val.cpu())

            val_loss += _loss.sum()

    f1_score_val = f1_score(torch.cat(ytrue_f1).numpy(), torch.cat(ypred_f1).numpy(), average='macro')

    print(f"Val Loss: {val_loss}")
    print(f"=" * 80)
    print(f"F1 score(val) {f1_score_val}")
    print(f"=" * 80)

  0%|          | 0/169 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([12, 1, 128, 1000])
self.norm(x) torch.Size([12, 768, 12, 99])
 patch_embed :  torch.Size([12, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X flattened torch.Size([12, 1188, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([12, 1, 768])
 self.dist_token.shape torch.Size([12, 1, 768])
 final sequence x torch.Size([12, 1190, 768])
 after 12 atten blocks x torch.Size([12, 1190, 768])
forward_features torch.Size([12, 768])
head torch.Size([12, 66])


100%|██████████| 169/169 [03:09<00:00,  1.12s/it]

Val Loss: 1069.6846923828125
F1 score(val) 0.8392231749961314


In [15]:
%%sh
nvidia-smi

Fri Jul 14 20:22:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   48C    P0    26W /  70W |  14666MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## Miscellaneous

### Glossary

SWA - Stochastic weight averaging

### Model stored weights
```
Downloading: "https://github.com/kkoutini/PaSST/releases/download/v0.0.1-audioset/passt-s-f128-p16-s10-ap.476-swa.pt" to /root/.cache/torch/hub/checkpoints/passt-s-f128-p16-s10-ap.476-swa.pt
```